Below are parameters of the experiments reported in [KLNO24](https://eprint.iacr.org/2024/1972).

In [29]:
from rok_estimator.rok_estimator import *


class JL:
    kjl = 36
    njl = 100
    mjl = kjl * njl
    def get_delta(self):
        return 2^(-self.njl)

    def get_right_bound(self):
        return sqrt(self.mjl * self.njl)
    def get_left_bound(self):
        return 1 / (2 * sqrt(self.mjl))


class HJL: # heuristics from GHL, BS
    kjl = 34
    njl = 256
    mjl = kjl * njl
    def get_delta(self):
        return 2^(-128)

    def get_right_bound(self):
        return sqrt(337)
    def get_left_bound(self):
        return sqrt(30)



jl = HJL()

print("Limits of structure: ", jl.mjl)


g_first_branch = None
g_second_branch = None

g_second_branch_ext_2_norm = None
g_second_branch_ext_inf_norm = None

class Relation_RP(Relation):
    def execute(self, op, **kwargs):
        match op:
            case "srp":
                return self.pi_srp(**kwargs)
            case "to-first":
                return self.pi_to_first(**kwargs)
            case "join":
                return self.pi_join(**kwargs)
            case "norm-poly":
                return self.pi_norm_poly()
            case _:
                return super().execute(op, **kwargs)
            
    def pi_norm_poly(self):
        comm = (3 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.size_Rq()  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
        # comm = (2 * self.rep) * self.ring.size_Rq() + (1 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.log_q  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
        snd_err = 3 * ceil(log(self.wdim, 2))/(2**(self.ring.log_q * self.ring.residue_deg)) # TODO: Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the soundness error.
        rel_params = {
            # "ring": self.ring,
            # "trivial": self.trivial,
            "op_name": "norm-poly",
            "n_compress": self.n_compress + 2,
            # "n_commit": self.n_commit,
            "n_rel": self.n_rel + 2,
            # "wdim": self.wdim,
            # "rep": self.rep,
            # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
            # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
            # "log_beta_wit_inf": self.log_beta_wit_inf
            "comm" : comm,      
            "acc_comm" : self.acc_comm + comm,       
            "snd_err" : snd_err,
            "acc_snd_err" : self.acc_snd_err + snd_err,           
            "log_beta_ext_2_func" : lambda x : self.log_beta_wit_2, # perfect extraction
            "log_beta_ext_inf_func" : lambda x : oo # extraction of ell_2-norm is perfect, then bound ell_inf-norm by norm conversion
        }      
        return replace(self, **rel_params)
            
    def pi_srp(self):
        global g_first_branch 
        rel_params = {
            "n_rel": self.n_rel + 1,
            "log_slack_2_func": lambda x: x,
            "log_slack_inf_func": lambda x: x, 
        }

        g_first_branch = replace(deepcopy(self), **rel_params)
        comm = (self.n_commit + 1) * self.ring.size_Rq()  # for the new comm
        snd_err = self.wdim * self.rep / jl.kjl * (1 / 2**(self.ring.log_q * self.ring.residue_deg) + jl.get_delta() * euler_phi(self.ring.f) / jl.njl)
        print(self.wdim)
        global g_second_branch_ext_2_norm
        global g_second_branch_ext_inf_norm

        def log_beta_ext_2_func(x):
            global g_second_branch_ext_2_norm
            print(radical(self.ring.f))
            return g_second_branch_ext_2_norm + log(1 / jl.get_left_bound(), 2) + log(sqrt(radical(self.ring.f)),2)
            
        def log_beta_ext_inf_func(x):
            global g_second_branch_ext_inf_norm # can we say sth about infinity - norm?
            return oo # TODO
            
        rel_params = {
            "op_name": "srp 2 rel",
            "n_compress": self.n_commit + 1,
            "n_rel": 1,
            "wdim": ZZ(ceil(self.wdim / jl.kjl * self.rep)),
            "rep": 1,
            "comm": comm,      
            "acc_comm" : self.acc_comm + comm,       
            "snd_err" : snd_err,
            "acc_snd_err" : self.acc_snd_err + snd_err,    
            "log_beta_wit_2": self.log_beta_wit_2 + log(jl.get_right_bound(), 2) + log(sqrt(self.rep),2),
            "log_beta_wit_inf":self.log_beta_wit_2 + log(jl.get_right_bound(), 2) + log(sqrt(self.rep),2), # can we say sth about infinity - norm?
            "log_beta_ext_2_func": log_beta_ext_2_func, # we extract norm from the second branch (relax it)
            "log_beta_ext_inf_func" : log_beta_ext_2_func,  # can we say sth about infinity - norm?
            "log_slack_2_func": lambda x: 0,
            "log_slack_inf_func": lambda x: 0,
        }     
        
        # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
        return replace(self, **rel_params)


    def pi_to_first(self):
        global g_second_branch 
        g_second_branch = deepcopy(self)
        
        rel_params = {
            "op_name": "    1 rel",
            "n_rel": 1,
            "log_beta_ext_2_func" : lambda x : x, # perfect extraction
            "log_beta_ext_inf_func" : lambda x : x, # perfect extraction
            # "log_slack_2_func" : lambda x: 0, # perfect extraction
            # "log_slack_inf_func" : lambda x: 0 # perfect extraction
        }      
        global g_first_branch
        return replace(g_first_branch, **rel_params)
        
    def pi_join(self):
        def log_beta_ext_2_func(x):
            global g_second_branch_ext_2_norm
            g_second_branch_ext_2_norm = x
            return x
        def log_beta_ext_inf_func(x):
            global g_second_branch_ext_inf_norm
            g_second_branch_ext_inf_norm = x
            return x

        global g_second_branch

        # assert g_second_branch.wdim <= self.wdim
        assert g_second_branch.rep == 1
        assert self.rep == 1
        
        rel_params = {
            "op_name": "join",
            "rep": 2,
            "n_rel": self.n_rel + g_second_branch.n_rel,
            "log_beta_wit_2": max(self.log_beta_wit_2, g_second_branch.log_beta_wit_2),
            "log_beta_wit_inf": max(self.log_beta_wit_inf, g_second_branch.log_beta_wit_inf),
            "log_beta_ext_2_func": log_beta_ext_2_func, # perfect extraction, but we keep track of what was there
            "log_beta_ext_inf_func" : log_beta_ext_inf_func # perfect extraction, but we keep track of what was there
        }      
        return replace(self, **rel_params)
        
        
f = 60
K = CyclotomicField(f)
phi = euler_phi(f)
R = K.ring_of_integers()

max_v = 100


C = ChallengeSet(cardinality = max_v^phi, gamma_2 = max_v * sqrt(phi) / 2, theta_2 = max_v * sqrt(phi), gamma_inf = max_v / 2, theta_inf = max_v)

Limits of structure:  8704


In [40]:
ring_params = {
    "f": 60,
    "log_beta_sis_2": 72,
    "log_q": 80,
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(4) * 13 + [("finish", {})]

sim = Simulation(ring_params, rel_params, Relation_RP)
sim.ring.C = C


sim.execute(ops)
sim.extract()
sim.show()

268697600
134348800
67174400
33587200
16793600
8396800
4198400
2099200
1049600
524800
262400
131200
65600
32800
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      | 268697600 |   1 |    ( 33 | 33/  0 )        |     ( 14 | 33/  0 )         | 7688. MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   | 268697600 |   2 |    ( 25 | 26/  0 )        |     (  7 | 26/  0 )         | 8200. MB |      (24.69 KB | 24.69 KB)      |         (2^-oo  | 2^-oo )         
 norm      | 268697600 |   8 |    ( 25 | 51/  0 )        |     (  7 | 51/  0 )         | 32800. MB |      (151.9 KB | 176.6 KB)      |         (2^-157 | 2^-157)         
 batch     | 268697600 |   8 |    ( 25 | 51/  0 )        |     (  7 | 51/  0 )         | 32800. MB |      (151.

In [41]:
ring_params = {
    "f": 60,
    "log_beta_sis_2": 36.9,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**18 * 41,
    "rep": 25,
    "log_beta_wit_inf": 10,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      | 10747904 |  25 |    ( 27 | 30/  0 )        |     ( 10 | 30/  0 )         | 5638. MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp   | 10747904 | 100 |    ( 18 | 22/  0 )        |     (  2 | 22/  0 )         | 6150. MB |      (487.5 KB | 487.5 KB)      |         (2^-oo  | 2^-oo )         
 norm      | 10747904 | 111 |    ( 18 | 36/  0 )        |     (  2 | 36/  0 )         | 6826. MB |      (113.1 KB | 600.6 KB)      |         (2^-124 | 2^-124)         
 batch     | 10747904 | 111 |    ( 18 | 36/  0 )        |     (  2 | 36/  0 )         | 6826. MB |      (113.1 KB | 600.6 KB)      |         (2^-119 | 2^-119)         
 split     |  5373952 | 222 |    ( 18 | 36/  0 )        |     (  2 | 36/  0 )         | 6826. MB |      (763.1 KB | 1.332 MB)      |         (2